<a href="https://colab.research.google.com/github/Filarh/Cringe-scraper/blob/main/Defooocuss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title DeFooocus
#@markdown **Launch the interface DeFocus (Fooocus fork)** | You need to connect with T4/A100/V100
#@markdown ****
#@markdown *Attention!* When working in the interface with the FaceSwap and CPDS controlnet, crashes are possible; it is also recommended to work in *Extreme speed* mode for additional stability. When working with the ImagePrompt and PyraCanny controls, 85% of the work will be stable.
#@markdown ****

print("[DeFooocus] Preparing ...")

theme = "dark" #@param ["dark", "light"]
preset = "deafult" #@param ["deafult", "realistic", "anime", "lcm", "sai", "turbo", "lighting", "hypersd", "playground_v2.5", "dpo", "spo", "sd1.5"]
advenced_args = "--share --attention-split --always-high-vram --disable-offload-from-vram --all-in-fp16" #@param {type: "string"}

if preset != "deafult":
  args = f"{advenced_args} --theme {theme} --preset {preset}"
else:
  args = f"{advenced_args} --theme {theme}"

!pip install -q pygit2==1.12.2
%cd /content
!git clone https://github.com/ehristoforu/DeFooocus.git
%cd /content/DeFooocus
!pip install -q -r requirements_versions.txt

# Importa la librería de Google Colab
import google.colab
import os
import json

# Check if Google Colab Drive needs to be activated
activar_drive = True #@param {type: "boolean"}

if activar_drive:
    from google.colab import drive
    drive.mount('/content/drive')
    # Desired paths
    nuevo_dir_lora = '/content/drive/MyDrive/MyLoras' #@param {type: "string"}
    nuevo_dir_salida = '/content/drive/MyDrive/Fooocus_output' #@param {type: "string"}

    # launch.py file
    launch_py_path = '/content/DeFooocus/launch.py'

    # Check if the file exists
    if os.path.exists(launch_py_path):
        # Open the launch.py file
        with open(launch_py_path, 'r') as file:
            launch_py_content = file.read()

        # Code to insert at the beginning of launch.py
        codigo_a_insertar = f'''
import os
import json

# Updated paths
nuevo_dir_lora = '{nuevo_dir_lora}'
nuevo_dir_salida = '{nuevo_dir_salida}'

# Configuration file
config_path = os.path.abspath("./config.txt")
config_dict = {{}}

if os.path.exists(config_path):
    with open(config_path, "r", encoding="utf-8") as json_file:
        config_dict = json.load(json_file)

# Modify paths in the configuration dictionary
config_dict['path_loras'] = os.path.abspath(nuevo_dir_lora)
config_dict['path_outputs'] = os.path.abspath(nuevo_dir_salida)

# Save changes to the configuration file
with open(config_path, "w", encoding="utf-8") as json_file:
    json.dump(config_dict, json_file, indent=4)

'''

        # Write the new content at the beginning of launch.py
        with open(launch_py_path, 'w') as file:
            file.write(codigo_a_insertar + launch_py_content)
    else:
        print("The launch.py file does not exist in the provided location.")

#@markdown -------------
# Crea una sección para modificar el modelo por defecto
#@markdown ### **Modificación del modelo por defecto**
#@markdown Indica si quieres modificar el modelo por defecto o no. Si lo haces, proporciona el nombre del archivo, el link del archivo y el aspect ratio que deseas usar.
modificar_modelo = True #@param {type: "boolean"}
nombre_archivo = "zavychromaxl"  #@param {type: "string"}
link_archivo = "https://huggingface.co/rafationgson/zavychroma-xl-v4/resolve/main/zavychromaxl_v40.safetensors?download=true"  #@param {type: "string"}
aspect_ratio = "1152*896" #@param {type: "string"}

# Define una función para modificar el archivo JSON con los parámetros proporcionados
def modificar_json(nombre_archivo: str, link_archivo: str, aspect_ratio: str):
    carpeta = '/content/DeFooocus/presets'
    archivos = os.listdir(carpeta)
    json_encontrado = next((archivo for archivo in archivos if archivo.endswith('.json')), None)

    if json_encontrado:
        with open(os.path.join(carpeta, json_encontrado)) as file:
            data = json.load(file)

        # Modificar los valores necesarios
        data['default_model'] = nombre_archivo + ".safetensors"  # Agregar la extensión al nombre del modelo
        data['default_aspect_ratio'] = aspect_ratio
        data['default_refiner'] = ""

        # Actualizar el diccionario checkpoint_downloads con el nuevo archivo y link
        data['checkpoint_downloads'] = {nombre_archivo + ".safetensors": link_archivo}  # Agregar la extensión al nombre del modelo

        # Establecer el argumento para la segunda celda basado en el nombre del archivo
        global args
        args += f" --preset {nombre_archivo}"  # Mantener solo el nombre del archivo sin extensión

        # Simplificar el nombre del archivo de salida
        nuevo_nombre = f"{nombre_archivo}.json"
        with open(os.path.join(carpeta, nuevo_nombre), 'w') as file:
            json.dump(data, file, separators=(',', ':'))

        return f"¡Se ha creado el archivo {nuevo_nombre} con los parámetros proporcionados y estructura correcta!"

    else:
        return "No se encontraron archivos JSON en la carpeta especificada."

# Si la variable es verdadera, llama a la función para modificar el JSON
if modificar_modelo:
  resultado = modificar_json(nombre_archivo, link_archivo, aspect_ratio)
  print(resultado)

#@markdown -------------
# Crea una sección para ejecutar el archivo entry_with_update.py con el argumento opcional para compartir los resultados
#@title Ejecución de entry_with_update.py
print("[DeFooocus] Starting ...")
!python entry_with_update.py $args